# Spaceship Titanic - V9 (V5 + GroupSpend_max_log + 5-Model Ensemble)

**History:**
- V1: CV 0.8265 / LB 0.8020 (49 features, overfit)
- V2: CV 0.8141 / LB 0.8071 (29 features, best LB baseline)
- V3: CV 0.8184 / LB 0.8041 (56 features, TE on interactions = overfit)
- V4: CV 0.8146 / LB 0.8055 (33 features, V2+simple EDA)
- V5: CV 0.8177 / LB **0.8087** (32 features, BEST LB)
- V6: CV 0.8165 / LB 0.8038 (Optuna = overfit on small data)
- V7: CV 0.8159 / LB 0.8038 (Stacking, no help)
- V8: CV 0.8211 / LB TBD (39 features, 5-model ensemble)

**V9 Strategy:** Surgical feature addition from V8 analysis:
- **Base**: V5 (32 features, best LB 0.80874)
- **+1 feature**: `GroupSpend_max_log` — rank 7 in V8 importance (only V8 new feature in top 10)
- **5-model ensemble**: LGB + XGB + CatBoost + ExtraTrees + RandomForest (V2 manual params)
- Total: **33 features**, diverse ensemble for better generalization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
import warnings, os

warnings.filterwarnings('ignore')

SEED = 42
N_FOLDS = 10
TARGET = 'Transported'

def seed_everything(seed=SEED):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()
print('V9 Setup complete.')

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample_sub = pd.read_csv('../data/sample_submission.csv')

print(f'Train: {train.shape}, Test: {test.shape}')

train['is_train'] = 1
test['is_train'] = 0
test[TARGET] = np.nan
df = pd.concat([train, test], axis=0, ignore_index=True)

spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
print(f'Combined: {df.shape}')

## Feature Engineering V9 = V2 Base (identical to V5)

In [ ]:
# === V2 Feature Engineering (IDENTICAL copy) ===

# PassengerId
df['_Group'] = df['PassengerId'].str.split('_').str[0].astype(int)
df['GroupSize'] = df.groupby('_Group')['PassengerId'].transform('count')
df['IsAlone'] = (df['GroupSize'] == 1).astype(int)

# Cabin
df['Deck'] = df['Cabin'].str.split('/').str[0]
df['CabinNum'] = df['Cabin'].str.split('/').str[1].astype(float)
df['Side'] = df['Cabin'].str.split('/').str[2]
df['CabinRegion'] = (df['CabinNum'] // 100).astype(float)

# Name
df['Surname'] = df['Name'].str.split().str[-1]
df['FamilySize'] = df.groupby('Surname')['PassengerId'].transform('count')
df.loc[df['Surname'].isna(), 'FamilySize'] = 1

# Booleans
df['CryoSleep'] = df['CryoSleep'].map({True: 1, False: 0, 'True': 1, 'False': 0})
df['VIP'] = df['VIP'].map({True: 1, False: 0, 'True': 1, 'False': 0})

print('Parsing done.')

In [ ]:
# === V2 Imputation (identical) ===
for col in spend_cols:
    mask = (df['CryoSleep'] == 1) & (df[col].isna())
    df.loc[mask, col] = 0

mask = (df['CryoSleep'].isna()) & (df[spend_cols].sum(axis=1) == 0)
df.loc[mask, 'CryoSleep'] = 1
mask = (df['CryoSleep'].isna()) & (df[spend_cols].sum(axis=1) > 0)
df.loc[mask, 'CryoSleep'] = 0

for col in df.select_dtypes(include=[np.number]).columns:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

for col in df.select_dtypes(include=['object']).columns:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

print(f'Imputation done. Nulls: {df.isnull().sum().sum()}')

In [ ]:
# === V2 Spending features (identical) ===
df['TotalSpend'] = df[spend_cols].sum(axis=1)
df['TotalSpend_log'] = np.log1p(df['TotalSpend'])
df['NoSpend'] = (df['TotalSpend'] == 0).astype(int)
df['NumServicesUsed'] = (df[spend_cols] > 0).sum(axis=1)

for col in spend_cols:
    df[f'{col}_log'] = np.log1p(df[col])

df['LuxurySpend'] = np.log1p(df['Spa'] + df['VRDeck'] + df['RoomService'])
df['BasicSpend'] = np.log1p(df['FoodCourt'] + df['ShoppingMall'])

# Age
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 5, 12, 18, 30, 50, 80],
                        labels=[0, 1, 2, 3, 4, 5]).astype(float)
df['IsChild'] = (df['Age'] < 18).astype(float)

# Interactions
df['CryoSleep_NoSpend'] = ((df['CryoSleep'] == 1) & (df['TotalSpend'] == 0)).astype(int)

# Group spending
df['GroupSpend_mean'] = df.groupby('_Group')['TotalSpend'].transform('mean')
df['GroupSpend_mean_log'] = np.log1p(df['GroupSpend_mean'])

# Label encoding
for col in ['HomePlanet', 'Destination', 'Deck', 'Side']:
    le = LabelEncoder()
    df[col + '_le'] = le.fit_transform(df[col].astype(str))

# Frequency encoding
for col in ['HomePlanet', 'Destination', 'Deck', 'Side']:
    freq = df[col].value_counts(normalize=True)
    df[col + '_freq'] = df[col].map(freq)

print('V2 features done.')

## V5 Target Encoding (Bayesian smoothed, 10-fold OOF)

In [ ]:
# === REGULARIZED TARGET ENCODING ===

te_cols = ['HomePlanet', 'Destination', 'Deck']
SMOOTHING = 20

train_idx = df[df['is_train'] == 1].index
test_idx = df[df['is_train'] == 0].index

y_for_te = df.loc[train_idx, TARGET].map({True: 1.0, False: 0.0, 1: 1.0, 0: 0.0}).astype(float)
df.loc[train_idx, '_target_float'] = y_for_te.values
global_mean = float(y_for_te.mean())

print(f'Global mean: {global_mean:.4f}')
print(f'Smoothing factor: {SMOOTHING}')

kf_te = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for col in te_cols:
    col_te = f'{col}_te'
    df[col_te] = global_mean
    
    for fold_train, fold_val in kf_te.split(train_idx, y_for_te):
        fold_train_idx = train_idx[fold_train]
        fold_val_idx = train_idx[fold_val]
        stats = df.loc[fold_train_idx].groupby(col)['_target_float'].agg(['mean', 'count'])
        stats['te'] = (stats['count'] * stats['mean'] + SMOOTHING * global_mean) / (stats['count'] + SMOOTHING)
        mapped = df.loc[fold_val_idx, col].map(stats['te']).fillna(global_mean)
        df.loc[fold_val_idx, col_te] = mapped.values
    
    stats = df.loc[train_idx].groupby(col)['_target_float'].agg(['mean', 'count'])
    stats['te'] = (stats['count'] * stats['mean'] + SMOOTHING * global_mean) / (stats['count'] + SMOOTHING)
    mapped = df.loc[test_idx, col].map(stats['te']).fillna(global_mean)
    df.loc[test_idx, col_te] = mapped.values
    
    print(f'{col}_te values:')
    print(df.loc[train_idx].groupby(col)[col_te].mean().sort_values())
    print()

df.drop('_target_float', axis=1, inplace=True)
print('Target encoding done.')

## V9 New Feature: GroupSpend_max_log

**Rank 7** in V8 LightGBM importance (importance=350.7). The ONLY V8 new feature in the top 10.

Captures: the maximum spending of any member in the travel group → strong signal for group behavior.

In [ ]:
# === V9 NEW FEATURE: GroupSpend_max_log ===
df['GroupSpend_max'] = df.groupby('_Group')['TotalSpend'].transform('max')
df['GroupSpend_max_log'] = np.log1p(df['GroupSpend_max'])

print(f'GroupSpend_max_log stats:')
print(df['GroupSpend_max_log'].describe())
print(f'\nCorrelation with GroupSpend_mean_log: {df["GroupSpend_max_log"].corr(df["GroupSpend_mean_log"]):.4f}')

In [ ]:
# === DEFINE FEATURES: V5 (32) + GroupSpend_max_log = 33 features ===
drop_cols = [
    'PassengerId', 'Name', 'Cabin', 'Surname', 'is_train', TARGET,
    'HomePlanet', 'Destination', 'Deck', 'Side',
    '_Group', 'CabinNum',
    'TotalSpend', 'GroupSpend_mean',
    'GroupSpend_max',  # keep only the log version
] + spend_cols

features = [c for c in df.columns if c not in drop_cols]
print(f'V9 features: {len(features)} (V5: 32 + 1 new = 33)')
print()
for i, f in enumerate(sorted(features)):
    marker = ' *** NEW V9' if f == 'GroupSpend_max_log' else (' [TE]' if f.endswith('_te') else '')
    print(f'  {i+1:2d}. {f}{marker}')

In [ ]:
# Split
train_df = df[df['is_train'] == 1].copy()
test_df = df[df['is_train'] == 0].copy()

X = train_df[features].values
y = train_df[TARGET].astype(int).values
X_test = test_df[features].values

print(f'X: {X.shape}, y: {y.shape}, X_test: {X_test.shape}')
print(f'Target mean: {np.mean(y):.4f}')

## Model 1: LightGBM (V2 manual params)

In [ ]:
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 20,
    'learning_rate': 0.03,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 5,
    'min_child_samples': 30,
    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'max_depth': 5,
    'n_estimators': 5000,
    'verbose': -1,
    'n_jobs': -1,
    'random_state': SEED,
}

oof_lgb = np.zeros(len(X))
test_lgb = np.zeros(len(X_test))
fi_lgb = np.zeros(len(features))

kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[tr_idx], X[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]
    
    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(500)]
    )
    
    oof_lgb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_lgb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    fi_lgb += model.feature_importances_ / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_lgb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

lgb_acc = accuracy_score(y, (oof_lgb > 0.5).astype(int))
print(f'\nLightGBM V9 CV: {lgb_acc:.5f}')

## Model 2: XGBoost (V2 manual params)

In [ ]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 4,
    'learning_rate': 0.03,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 10,
    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 0.1,
    'n_estimators': 5000,
    'early_stopping_rounds': 200,
    'tree_method': 'hist',
    'random_state': SEED,
    'verbosity': 0,
}

oof_xgb = np.zeros(len(X))
test_xgb = np.zeros(len(X_test))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[tr_idx], X[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]
    
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=500)
    
    oof_xgb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_xgb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_xgb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

xgb_acc = accuracy_score(y, (oof_xgb > 0.5).astype(int))
print(f'\nXGBoost V9 CV: {xgb_acc:.5f}')

## Model 3: CatBoost (V2 manual params)

In [ ]:
oof_cb = np.zeros(len(X))
test_cb = np.zeros(len(X_test))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[tr_idx], X[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]
    
    model = CatBoostClassifier(
        iterations=5000,
        learning_rate=0.03,
        depth=4,
        l2_leaf_reg=5.0,
        subsample=0.7,
        colsample_bylevel=0.7,
        min_data_in_leaf=30,
        random_seed=SEED,
        verbose=500,
        early_stopping_rounds=200,
        task_type='CPU',
    )
    
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val))
    
    oof_cb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_cb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_cb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

cb_acc = accuracy_score(y, (oof_cb > 0.5).astype(int))
print(f'\nCatBoost V9 CV: {cb_acc:.5f}')

## Model 4: ExtraTrees (diversity booster)

In [ ]:
oof_et = np.zeros(len(X))
test_et = np.zeros(len(X_test))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[tr_idx], X[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]
    
    model = ExtraTreesClassifier(
        n_estimators=2000,
        max_depth=12,
        min_samples_leaf=10,
        max_features=0.7,
        random_state=SEED,
        n_jobs=-1,
    )
    model.fit(X_tr, y_tr)
    
    oof_et[val_idx] = model.predict_proba(X_val)[:, 1]
    test_et += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_et[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

et_acc = accuracy_score(y, (oof_et > 0.5).astype(int))
print(f'\nExtraTrees V9 CV: {et_acc:.5f}')

## Model 5: RandomForest (diversity booster)

In [ ]:
oof_rf = np.zeros(len(X))
test_rf = np.zeros(len(X_test))

for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[tr_idx], X[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]
    
    model = RandomForestClassifier(
        n_estimators=2000,
        max_depth=12,
        min_samples_leaf=10,
        max_features=0.7,
        random_state=SEED,
        n_jobs=-1,
    )
    model.fit(X_tr, y_tr)
    
    oof_rf[val_idx] = model.predict_proba(X_val)[:, 1]
    test_rf += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_rf[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

rf_acc = accuracy_score(y, (oof_rf > 0.5).astype(int))
print(f'\nRandomForest V9 CV: {rf_acc:.5f}')

## Feature Importance

In [ ]:
fi_df = pd.DataFrame({'feature': features, 'importance': fi_lgb})
fi_df = fi_df.sort_values('importance', ascending=True).tail(20)

plt.figure(figsize=(10, 7))
colors = ['coral' if f == 'GroupSpend_max_log' else ('gold' if f.endswith('_te') else 'steelblue') 
          for f in fi_df['feature']]
plt.barh(fi_df['feature'], fi_df['importance'], color=colors)
plt.title(f'LightGBM V9 Feature Importance - CV: {lgb_acc:.5f}\n(coral=V9 new, gold=TE, blue=V2 base)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## Ensemble Analysis & Submissions

In [ ]:
print('=== V9 Individual Model Results ===')
print(f'LightGBM:    {lgb_acc:.5f}')
print(f'XGBoost:     {xgb_acc:.5f}')
print(f'CatBoost:    {cb_acc:.5f}')
print(f'ExtraTrees:  {et_acc:.5f}')
print(f'RandomForest:{rf_acc:.5f}')

# OOF correlations
print('\n=== OOF Correlations ===')
oof_all = pd.DataFrame({
    'LGB': oof_lgb, 'XGB': oof_xgb, 'CB': oof_cb, 'ET': oof_et, 'RF': oof_rf
})
print(oof_all.corr().round(4))

In [ ]:
# === ENSEMBLE STRATEGIES ===

# 1. 3-Model GBDT Average (like V5)
oof_3avg = (oof_lgb + oof_xgb + oof_cb) / 3
acc_3avg = accuracy_score(y, (oof_3avg > 0.5).astype(int))
test_3avg = (test_lgb + test_xgb + test_cb) / 3

# 2. 5-Model Equal Average
oof_5avg = (oof_lgb + oof_xgb + oof_cb + oof_et + oof_rf) / 5
acc_5avg = accuracy_score(y, (oof_5avg > 0.5).astype(int))
test_5avg = (test_lgb + test_xgb + test_cb + test_et + test_rf) / 5

# 3. 5-Model Weighted (GBDT heavier, ET/RF lighter for diversity)
w_lgb, w_xgb, w_cb, w_et, w_rf = 0.25, 0.25, 0.25, 0.125, 0.125
oof_5w = w_lgb*oof_lgb + w_xgb*oof_xgb + w_cb*oof_cb + w_et*oof_et + w_rf*oof_rf
acc_5w = accuracy_score(y, (oof_5w > 0.5).astype(int))
test_5w = w_lgb*test_lgb + w_xgb*test_xgb + w_cb*test_cb + w_et*test_et + w_rf*test_rf

# 4. Majority voting (5 models)
votes = ((oof_lgb > 0.5).astype(int) + (oof_xgb > 0.5).astype(int) + 
         (oof_cb > 0.5).astype(int) + (oof_et > 0.5).astype(int) + 
         (oof_rf > 0.5).astype(int))
acc_vote = accuracy_score(y, (votes >= 3).astype(int))

# 5. Optimized grid search on weights
best_acc = 0
best_weights = None
for w_gbdt in np.arange(0.20, 0.36, 0.01):
    for w_tree in np.arange(0.05, 0.21, 0.01):
        w_rem = 1.0 - 3*w_gbdt - 2*w_tree
        if abs(w_rem) > 0.01:  # must sum to ~1
            continue
        oof_try = w_gbdt*oof_lgb + w_gbdt*oof_xgb + w_gbdt*oof_cb + w_tree*oof_et + w_tree*oof_rf
        acc_try = accuracy_score(y, (oof_try > 0.5).astype(int))
        if acc_try > best_acc:
            best_acc = acc_try
            best_weights = (w_gbdt, w_tree)

if best_weights:
    w_g, w_t = best_weights
    oof_opt = w_g*oof_lgb + w_g*oof_xgb + w_g*oof_cb + w_t*oof_et + w_t*oof_rf
    acc_opt = accuracy_score(y, (oof_opt > 0.5).astype(int))
    test_opt = w_g*test_lgb + w_g*test_xgb + w_g*test_cb + w_t*test_et + w_t*test_rf
else:
    acc_opt = acc_5w
    test_opt = test_5w
    w_g, w_t = 0.25, 0.125

print('=== ENSEMBLE RESULTS ===')
print(f'3-Model GBDT Avg:       {acc_3avg:.5f}')
print(f'5-Model Equal Avg:      {acc_5avg:.5f}')
print(f'5-Model Weighted:       {acc_5w:.5f}  (GBDT 0.25 each, ET/RF 0.125 each)')
print(f'5-Model Majority Vote:  {acc_vote:.5f}')
print(f'5-Model Optimized:      {acc_opt:.5f}  (GBDT {w_g:.2f} each, ET/RF {w_t:.2f} each)')

print(f'\n=== VERSION COMPARISON ===')
print(f'V1: CV 0.82653 | LB 0.80196 | 49 features')
print(f'V2: CV 0.81410 | LB 0.80710 | 29 features')
print(f'V3: CV 0.81836 | LB 0.80406 | 56 features')
print(f'V4: CV 0.81456 | LB 0.80547 | 33 features')
print(f'V5: CV 0.81767 | LB 0.80874 | 32 features (BEST LB)')
print(f'V8: CV 0.82112 | LB TBD     | 39 features')
print(f'V9: CV {acc_opt:.5f} | LB TBD     | {len(features)} features (V5 + 1 feature + 5 models)')

In [ ]:
# === SUBMISSIONS ===

# Primary: best ensemble (optimized weights)
final_preds = (test_opt > 0.5)
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'].values,
    'Transported': final_preds
})
submission['Transported'] = submission['Transported'].astype(bool)
submission.to_csv('../submissions/submission_v9.csv', index=False)

assert submission.shape[0] == sample_sub.shape[0]
assert list(submission.columns) == list(sample_sub.columns)

print(f'V9 Primary (Optimized 5-model): saved to submissions/submission_v9.csv')
print(f'  True: {final_preds.sum()}, False: {(~final_preds).sum()}, Ratio: {final_preds.mean():.4f}')

# Safe fallback: 3-model GBDT average (closest to V5 which had best LB)
final_safe = (test_3avg > 0.5)
submission_safe = pd.DataFrame({
    'PassengerId': test_df['PassengerId'].values,
    'Transported': final_safe
})
submission_safe['Transported'] = submission_safe['Transported'].astype(bool)
submission_safe.to_csv('../submissions/submission_v9_3model.csv', index=False)

print(f'\nV9 Safe (3-GBDT avg): saved to submissions/submission_v9_3model.csv')
print(f'  True: {final_safe.sum()}, False: {(~final_safe).sum()}, Ratio: {final_safe.mean():.4f}')

# How many predictions differ between the two?
diff = (final_preds != final_safe).sum()
print(f'\nDifferences between 5-model and 3-model: {diff} ({diff/len(final_preds)*100:.1f}%)')